In [42]:
import requests  #to query the API 
import re  #regular expressions
import pandas as pd   # for dataframes
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.model_selection import train_test_split
import string
from nltk.stem.snowball import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
endpoint="https://newsapi.org/v2/everything"

In [4]:
topics=["crime", "jobs", "cars", "weather"]

In [8]:
filename="NewHeadlines.csv"
MyFILE=open(filename,"w")
WriteThis="LABEL,Date,Source,Title,Headline\n"
MyFILE.write(WriteThis)
MyFILE.close()

In [22]:
for topic in topics:

    ## Dictionary Structure
    URLPost = {'apiKey':'8f4134f7d0de43b8b49f91e22100f22b',
               'q':topic
    }

    response=requests.get(endpoint, URLPost)
    print(response)
    jsontxt = response.json()
    print(jsontxt)
    #####################################################
    
    
    ## Open the file for append
    MyFILE=open(filename, "a")
    LABEL=topic
    for items in jsontxt["articles"]:
        print(items, "\n\n\n")
                  
        #Author=items["author"]
        #Author=str(Author)
        #Author=Author.replace(',', '')
        
        Source=items["source"]["name"]
        print(Source)
        
        Date=items["publishedAt"]
        ##clean up the date
        NewDate=Date.split("T")
        Date=NewDate[0]
        print(Date)
        
        ## CLEAN the Title
        ##----------------------------------------------------------
        ##Replace punctuation with space
        # Accept one or more copies of punctuation         
        # plus zero or more copies of a space
        # and replace it with a single space
        Title=items["title"]
        Title=str(Title)
        #print(Title)
        Title=re.sub(r'[,.;@#?!&$\-\']+', ' ', str(Title), flags=re.IGNORECASE)
        Title=re.sub(' +', ' ', str(Title), flags=re.IGNORECASE)
        Title=re.sub(r'\"', ' ', str(Title), flags=re.IGNORECASE)
        
        # and replace it with a single space
        ## NOTE: Using the "^" on the inside of the [] means
        ## we want to look for any chars NOT a-z or A-Z and replace
        ## them with blank. This removes chars that should not be there.
        Title=re.sub(r'[^a-zA-Z]', " ", str(Title), flags=re.VERBOSE)
        Title=Title.replace(',', '')
        Title=' '.join(Title.split())
        Title=re.sub("\n|\r", "", Title)
        print(Title)
        ##----------------------------------------------------------
        
        Headline=items["description"]
        Headline=str(Headline)
        Headline=re.sub(r'[,.;@#?!&$\-\']+', ' ', Headline, flags=re.IGNORECASE)
        Headline=re.sub(' +', ' ', Headline, flags=re.IGNORECASE)
        Headline=re.sub(r'\"', ' ', Headline, flags=re.IGNORECASE)
        Headline=re.sub(r'[^a-zA-Z]', " ", Headline, flags=re.VERBOSE)
        ## Be sure there are no commas in the headlines or it will
        ## write poorly to a csv file....
        Headline=Headline.replace(',', '')
        Headline=' '.join(Headline.split())
        Headline=re.sub("\n|\r", "", Headline)
        
        ### AS AN OPTION - remove words of a given length............
        Headline = ' '.join([wd for wd in Headline.split() if len(wd)>3])
    
        #print("Author: ", Author, "\n")
        #print("Title: ", Title, "\n")
        #print("Headline News Item: ", Headline, "\n\n")
        
        #print(Author)
        print(Title)
        print(Headline)
        
        WriteThis=str(LABEL)+","+str(Date)+","+str(Source)+","+ str(Title) + "," + str(Headline) + "\n"
        print(WriteThis)
        
        MyFILE.write(WriteThis)
        
    ## CLOSE THE FILE
    MyFILE.close()
    

<Response [200]>
{'status': 'ok', 'totalResults': 33537, 'articles': [{'source': {'id': 'engadget', 'name': 'Engadget'}, 'author': 'Will Shanklin', 'title': "Vrbo's Super Bowl 'risk scores' are basically pre-crime for house parties", 'description': 'As the Super Bowl approaches, Airbnb rival Vrbo announced its use of “unauthorized event prevention technology,” algorithms that try to avoid house parties at rental properties. Similar to a credit system, the tech generates a risk score for each booking, giv…', 'url': 'https://www.engadget.com/vrbo-super-bowl-house-party-risk-score-193020734.html', 'urlToImage': 'https://s.yimg.com/os/creatr-uploaded-images/2023-01/24a52e70-a189-11ed-bedb-e4a63aac0e3a', 'publishedAt': '2023-01-31T18:25:20Z', 'content': 'As the Super Bowl approaches, Airbnb rival Vrbo announced its use of unauthorized event prevention technology, algorithms that try to avoid house parties at rental properties. Similar to a credit sys… [+1138 chars]'}, {'source': {'id': 'eng

In [38]:
df = pd.read_csv("Headlines.csv.csv")
df = df.head(13)
df.drop(labels = ['Unnamed: 1','Unnamed: 2','Unnamed: 3'], axis =1, inplace = True)

In [40]:
#Converting review column to lower case
df['Headline'] = df['Headline'].str.lower()

In [43]:
#removing punctuations
def remove_punctuation(text):
  for element in text:
    if element in string.punctuation:
      text = text.replace(element, "")
  return text
df['Headline'] = df['Headline'].apply(remove_punctuation)

In [45]:
#removing stopwords
stop =stopwords.words('english')
df['Headline'] = df['Headline'].apply( lambda x : " ".join([word for word in x.split() if word not in (stop)]))

In [46]:
#Stemming
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')
df['Headline'] = df['Headline'].apply(lambda x: " ".join([stemmer.stem(word) for word in x.split()]))

In [48]:
# cleaned data frame
df

,LABEL,Headline
0,crime,super bowl approach airbnb rival vrbo announc ...
1,crime,paramount report join trend stream platform re...
2,crime,special polic squad unit focus violent crime c...
3,crime,readi meet girl genius lunella lafayett partne...
4,crime,unpreced layoff ten thousand high qualifi comp...
5,crime,janic hallett mysteri pair post apocalypt lock...
6,jobs,economist bet labor market cool decemb music i...
7,jobs,latest month job report releas expect show eco...
8,jobs,googl becom latest tech giant announc round la...
9,jobs,amazon cut job previous plan andi jassi admit ...


In [49]:
#seperating labels
X = df['Headline']
y = df['LABEL']

In [54]:
#Test Train split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [56]:
#bag of words representation
#count_vect = CountVectorizer()
#bow_headline = count_vect.fit_transform(X)
#bow_test = count_vect.transform(X_test) # we cannot use fit_tranform here as it will introduce bias into our testing data

In [ ]:
#vecotrizing text
tf_vectorizer = CountVectorizer(max_features=50,stop_words='english')

#getting the TF matrix
tf = tf_vectorizer.fit_transform(df.pop('Headline'))

# adding "features" columns as SparseSeries
for i, col in enumerate(tf_vectorizer.get_feature_names()):
    df[col] = pd.SparseSeries(tf[:, i].toarray().ravel(), fill_value=0)

In [61]:
import scipy.sparse    
pd.DataFrame.sparse.from_spmatrix(tf)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,0,1,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
5,0,0,0,0,0,0,0,0,2,2,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,1,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,1,0,0,0,0,1,1,0,0,0,...,0,1,1,0,0,0,0,0,0,0
9,1,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
